<h1><center> About the MNIST dataset</center></h1>

![](https://cdn-images-1.medium.com/max/1600/1*1TkgO9Zz6rC3KpAYNl5KfA.png)

<h1>What is the MNIST dataset?</h1>

The MNIST dataset(<b>M</b>odified <b>N</b>ational <b>I</b>nstitute of <b>S</b>tandards and <b>T</b>echnology database)is a large dataset containing images of handwritten digits, similar to the ones above. It's most commonly used for trainning image processing systems and algorithims. It's based on the <i>NIST</i> dataset and was edited to just be better in general. Some of these changes include the people who wrote the digits as well as normalizing a lot of the digits. The website for the dataset can be found [here](http://yann.lecun.com/exdb/mnist/).

# Details
* 60,000 training images and 10,000 testing images.
* Digits written by high school students and employees of the United States Census Bureau.

![](https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png)

<h1> 1. Unzip the files</h1>


The pictures of the numbers are stored as .idx files in a zipped folder. As such, we are going to have to figure out how to handle these files. We need to unzip the folders to get at the data inside as well as figuring out how we are going to read the numbers in a meaningful way. Let's start by unzipping the folders.

<b>WARNING</b>
The files required for this notebook are not included. You should create a folder called 'data' and drop the zip files from [here](http://yann.lecun.com/exdb/mnist/) into it.
<b>WARNING</b>

The following files are compressed using gzip. Fortunately, Python has a library to help us deal with these gzip files.

In [1]:
# Import gzip to unzip our files
import gzip

# Using gzip we just import, open the zip files contained in our data folder
with gzip.open('data/t10k-labels-idx1-ubyte.gz', 'rb') as file:
    file_content = file.read()  

# Print the first 4 elements of our file to make sure we're opening it properly
file_content[0:4]

b'\x00\x00\x08\x01'

<h1> 2. Read the IDX files</h1>


Now, to read the IDX files contained in the zipped folders, we can use a package 'idx2numpy' which will convert our idx files into a numpy array.

In [2]:
# Impor the Python image library
import PIL

# Import numpy as np
import numpy as np

# Import cv2 to save the image
import cv2

image = ~np.array(list(file_content[16:800])).reshape(28,28).astype(np.uint8)

cv2.imwrite('firstImage.png', image)

True